In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy import stats

import ecephys_analyses as ea 
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
experiment = "sleep-homeostasis"
bsl_condition = "extended-wake-circadian-match"
rec_condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'
f_range = (0.5, 4)

In [6]:
def get_subject_data(
    subject, experiment, bsl_condition, rec_condition, states, duration, f_range
):
    rec_hyp = ea.load_hypnogram(subject, experiment, rec_condition)
    rec_hyp = rec_hyp.keep_states(states).keep_first(duration)

    rec_pow = (
        ea.load_power(subject, experiment, rec_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    rec_pow = (
        filter_dataset_by_hypnogram(rec_pow, rec_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )

    bsl_hyp = ea.load_hypnogram(subject, experiment, bsl_condition)
    bsl_hyp = bsl_hyp.keep_states(states).keep_first(duration)

    bsl_pow = (
        ea.load_power(subject, experiment, bsl_condition, "spg.nc")
        .sel(frequency=slice(*f_range))
        .sum(dim="frequency")
    )
    bsl_pow = (
        filter_dataset_by_hypnogram(bsl_pow, bsl_hyp)
        .swap_dims({"time": "datetime"})
        .drop_vars(["time", "timedelta"])
    )

    diff_means = rec_pow.mean() - bsl_pow.mean()
    diff_means_std = diff_means / bsl_pow.mean()
    diff_meds = rec_pow.median() - bsl_pow.median()
    diff_meds_std = diff_meds / bsl_pow.median()

    index = pd.MultiIndex.from_product(
        [[subject], ["cortex", "hippocampus"]], names=["subject", "region"]
    )
    data = np.asarray(
        [
            [
                diff_means.mpta_wm_ref,
                diff_means_std.mpta_wm_ref,
                diff_meds.mpta_wm_ref,
                diff_meds_std.mpta_wm_ref,
            ],
            [
                diff_means.sr_wm_ref,
                diff_means_std.sr_wm_ref,
                diff_meds.sr_wm_ref,
                diff_meds_std.sr_wm_ref,
            ],
        ]
    )
    return pd.DataFrame(
        data,
        columns=["diff_means", "diff_means_std", "diff_meds", "diff_meds_std"],
        index=index,
    )

In [7]:
subj_data = list()
for subject in subjects:
    subj_data.append(get_subject_data(subject, experiment, bsl_condition, rec_condition, states, duration, f_range))
df = pd.concat(subj_data)

In [8]:
print(df.to_markdown())

|                               |   diff_means |   diff_means_std |   diff_meds |   diff_meds_std |
|:------------------------------|-------------:|-----------------:|------------:|----------------:|
| ('Segundo', 'cortex')         |    171860    |        0.425483  |    193588   |        0.56492  |
| ('Segundo', 'hippocampus')    |     54153.9  |        0.119158  |     67816.6 |        0.174819 |
| ('Valentino', 'cortex')       |      4025.44 |        0.0261369 |     17600.1 |        0.170911 |
| ('Valentino', 'hippocampus')  |    120573    |        0.266149  |    135406   |        0.437357 |
| ('Doppio', 'cortex')          |    224659    |        0.708642  |    226019   |        0.902028 |
| ('Doppio', 'hippocampus')     |     99462    |        0.363665  |     87372.9 |        0.426615 |
| ('Alessandro', 'cortex')      |     59471.5  |        0.191313  |     85864.1 |        0.311886 |
| ('Alessandro', 'hippocampus') |     65512.9  |        0.131654  |    113784   |        0.294561 |


In [9]:
cx = df.xs('cortex', level=1, drop_level=False)
hc = df.xs('hippocampus', level=1, drop_level=False)

In [10]:
stats.ttest_1samp(cx, 0)

Ttest_1sampResult(statistic=array([2.43355484, 2.73216446, 2.71184491, 3.36643541]), pvalue=array([0.07170525, 0.0523261 , 0.05343524, 0.02813647]))

In [11]:
stats.ttest_1samp(hc, 0)

Ttest_1sampResult(statistic=array([6.6838557 , 4.92015637, 5.8842767 , 5.69066301]), pvalue=array([0.00260532, 0.00792851, 0.00416933, 0.0047097 ]))